In [10]:
import pandas as pd 
import numpy as np 
import infrastructure
import filepath 
import pygsheets
import datetime as dt
from datetime import datetime
import openpyxl 
import os
import gspread
from pathlib import Path
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)

# use creds to create a client to interact with the Google Drive API
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
mamba = gc.open_by_url('https://docs.google.com/spreadsheets/d/12vqSDueybprNphtsw7gXR5vmgcPG6_5ZNcnWzNpiasY/edit#gid=1238872091') 
pipeline_test = gc.open_by_url("https://docs.google.com/spreadsheets/d/16vrHMWs0ambcBJ1sC0SqpYVG8SOSWlbg1N22-bF49v8/edit#gid=1963352944")
pipeline_test_wks = pipeline_test.worksheet('title','Testing Scheduler')
pipeline_test_offer_info = pipeline_test.worksheet('title','Pub Info')
oi_df = pipeline_test_offer_info.get_as_df()
pl_df = pipeline_test_wks.get_as_df()
# open worksheet - mamba
schedule_wks  =  mamba.worksheet('title','New Mamba')
schedule_df = schedule_wks.get_as_df()
#la nina
lanina = infrastructure.get_lanina()
lanina1 =  lanina[~(lanina['Content Approval Status'].str.contains('Paused|Not Approved|Failed Testing', na = False)) ]

In [11]:
#Publisher Config
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
publisher_config = gc.open_by_url(
    'https://docs.google.com/spreadsheets/d/1Tzda6Djr3zQmOhWu7Ief3GVR9Cjaml8238CeX7chj_U/edit#gid=1620368362') 
pub_config = publisher_config[0].get_as_df()
pub_config.rename(columns={'DP.DS or DP.sV': 'DP.SV'}, inplace=True)

In [12]:
import smartsheet

# Smartsheet setup
smartsheet_token = '3g2rkftxrKUZfFaYx8roGBhXUKo1njSLp8vdX'
smart = smartsheet.Smartsheet(smartsheet_token)

# Smartsheet: Get data from a specific sheet
sheet_id = '795F6FwrFH953PC5V2PHCVP9FG5R7CphWVcQ5RQ1?view=grid&filterId=5028946508730244'
sheet = smart.Sheets.get_sheet(sheet_id)

INFO:smartsheet.smartsheet:{"request": {"command": "GET https://api.smartsheet.com/2.0/sheets/795F6FwrFH953PC5V2PHCVP9FG5R7CphWVcQ5RQ1?view=grid&filterId=5028946508730244"}}


In [13]:
def match_and_construct_full_string(input_string, df):
    # extract the parts of the input string
    parts = input_string.split('_')
    # Reconstruct the prefix including the second portion
    prefix_including_second_portion = '_'.join(parts[:2])
    
    # Find rows in the dataframe where DP.SV contains the second portion
    second_portion = parts[1]
    matching_rows = df[df['DP.SV'].astype(str).str.contains(second_portion, na=False)]
    
    # Check if any row is found
    if not matching_rows.empty:
        # Use the first matching row for this example
        first_match = matching_rows.iloc[0]
        # Construct the new string using the reconstructed prefix and the PUBID from the first matching row
        new_string = f"{prefix_including_second_portion}_{int(first_match['PUBID'])}"
        return new_string
    else:
        return "No matching PUBID found in the dataframe for the given portion."

In [14]:
def find_offer(df, dp_sv, hpid):
    code = dp_sv.split('_')[0]
    
    # Filter the DataFrame based on the code and check if the offer contains the extracted code
    filtered_df = df[df['SS Offers (updated)'].str.startswith(hpid) & df['SS Offers (updated)'].str.contains(code)]
    
    # Check if any row matches the criteria
    if not filtered_df.empty:
        return filtered_df['SS Offers (updated)'].iloc[0]
    else:
        return "No offer found."


In [15]:
column_map = {col.id: col.title for col in sheet.columns}

# Process the data
processed_data = []
for row in sheet.rows:
    date_format = '%-m/%d/%y'
    # Create a dictionary for each row mapping column names to cell values
    row_data = {column_map[cell.column_id]: cell.value for cell in row.cells if cell.column_id in column_map}

    # Check if the row has a "Pending" status in the 'Result' column
    if row_data.get('Result') == 'Pending':
        # Extract the required data for each drop
        for i in range(1, 4):  # Assuming up to 3 drops
            drop_date = row_data.get(f'Drop {i} Date')
            drop_date = dt.datetime.strptime(drop_date, '%Y-%m-%d').strftime(date_format)
            
            drop_segment = row_data.get(f'Drop {i} Dataset_Segment')
            dataset = match_and_construct_full_string(drop_segment, pub_config)
            drop_number = "2"
            time = "10:00 AM PST"
            send_strategy = "PT"
            if "OW" in str(row_data.get('Hitpath Offer ID')):
                hitpath_offer_id = row_data.get('Hitpath Offer ID')
            else:
                hitpath_offer_id = str(int(row_data.get('Hitpath Offer ID')))
            scheduling_name = find_offer(oi_df,dataset,hitpath_offer_id)
            
            offset = "0"
            creative = ""
            job_name = ""
            
            drop_limit = int(row_data.get(f'Drop {i} Limit'))
           
            # Only process rows where drop date is available
            if drop_date:
                processed_row = [drop_date, dataset, drop_number, time, drop_segment, send_strategy, scheduling_name,
                                 drop_limit, offset, creative, job_name]
                processed_data.append(processed_row)


In [16]:
processed_data

[['3/22/24',
  'MBC_PN.SWP_461500',
  '2',
  '10:00 AM PST',
  'MBC_PN.SWP_1PLD.30DC_TMO',
  'PT',
  '13070 - MBC',
  2600,
  '0',
  '',
  ''],
 ['3/22/24',
  'MBC_PN.FC_461653',
  '2',
  '10:00 AM PST',
  'MBC_PN.FC_21DC',
  'PT',
  '13070 - MBC',
  2600,
  '0',
  '',
  ''],
 ['3/22/24',
  'FLC_EDM.247L_461227',
  '2',
  '10:00 AM PST',
  'FLC_EDM.247L_30DC_NEXL',
  'PT',
  '13070 - FLC',
  2500,
  '0',
  '',
  '']]

In [17]:
new_rows = pd.DataFrame(processed_data, columns=[
    "Testing Date",  
    "Shortcode_DP.SV",  
    "Drop Number",  
    "Time",
    "Segment",
    "Send Strategy",  
    "Offer",
    "Limit", 
    "Offset",
    "Creative",
    "Job Name"
    
])

# Adjust data types to match original dataframe
new_rows["Drop Number"] = new_rows["Drop Number"].astype(int)
new_rows["Limit"] = new_rows["Limit"].astype(int)
new_rows["Offset"] = new_rows["Offset"].astype(int)


In [18]:
first_empty_row = len(pipeline_test_wks.get_col(1, include_tailing_empty=False)) + 1

# Append the DataFrame to the worksheet
pipeline_test_wks.set_dataframe(new_rows, (first_empty_row, 1), copy_head=False)  # (row, col)

In [19]:
pl_df = pipeline_test_wks.get_as_df()

In [20]:
# find the row index in the schedule sheet
def find_schedule_row(schedule_df, shortcode):
    matching_rows = schedule_df[schedule_df['Dataset'] == shortcode].index
    return matching_rows[0] if not matching_rows.empty else None

In [21]:
# find the column index for the date
def find_date_column(schedule_wks, test_date):
    for idx, col in enumerate(schedule_wks.get_row(2, include_tailing_empty=False)[3:], start=4):
        if pd.to_datetime(col).date() == test_date.date():
            return idx
    return None

In [22]:
def column_to_letters(column):
    """Convert a column number to a column letter."""
    string = ""
    while column > 0:
        column, remainder = divmod(column - 1, 26)
        string = chr(65 + remainder) + string
    return string

In [23]:
for index, row in pl_df.iterrows():  
    if len(row['Offer'].split()) == 4:
        sc = row['Offer'].split()[3]
        hitpath = row['Offer'].split()[0]
        df_content= lanina1[(lanina1['OfferIDs'] == hitpath) & (lanina1['Type'] == sc) & (lanina1['Channel'] =='SC')][['Reporting Content ID','Content Approval Status', 'Content']]  
        ccid  = df_content['Reporting Content ID'].values[0]
        col_idx = 10
        col_letter = column_to_letters(col_idx)
        cell_address = f"{col_letter}{index+2}"
        pipeline_test_wks.update_value(cell_address, ccid)
    elif len(row['Offer'].split()) == 3:
        sc = row['Offer'].split()[2]
        hitpath = row['Offer'].split()[0]
        df_content= lanina1[(lanina1['OfferIDs'] == int(hitpath)) & (lanina1['Type'] == sc) & (lanina1['Channel'] =='SC')][['Reporting Content ID','Content Approval Status', 'Content']]  
        ccid  = df_content['Reporting Content ID'].values[0]
        col_idx = 10
        col_letter = column_to_letters(col_idx)
        cell_address = f"{col_letter}{index+2}"
        pipeline_test_wks.update_value(cell_address, ccid)
    else:
        sc = row['Offer'].split()[4]
        hitpath = row['Offer'].split()[0]
        df_content= lanina1[(lanina1['OfferIDs'] == int(hitpath)) & (lanina1['Type'] == sc) & (lanina1['Channel'] =='SC')][['Reporting Content ID','Content Approval Status', 'Content']]  
        ccid  = df_content['Reporting Content ID'].values[0]
        col_idx = 10
        col_letter = column_to_letters(col_idx)
        cell_address = f"{col_letter}{index+2}"
        pipeline_test_wks.update_value(cell_address, ccid)

In [24]:
pl_df = pipeline_test_wks.get_as_df()

In [25]:
# Iterate over pl_df and update the schedule sheet
for index, row in pl_df.iterrows():
    shortcode = row['Shortcode_DP.SV']
    test_date = pd.to_datetime(row['Testing Date'])
    drop_number = row['Drop Number']
    time = row['Time']
    segment = row['Segment']
    send_strategy = row['Send Strategy']
    offer = row['Offer']
    limit = row['Limit']
    offset = row['Offset']
    
    creative = row['Creative']
    job_name = row['Job Name']
    p_limit = "Total - "+(str(limit+1))
    p_offset = str(limit+1)
    
    schedule_row_idx = find_schedule_row(schedule_df, shortcode)
    date_col_idx = find_date_column(schedule_wks, test_date)

    if schedule_row_idx is not None and date_col_idx is not None:
        values_to_update = [time, segment, send_strategy, offer, limit, offset, creative, job_name]

        # update cells with new data
        for i, value in enumerate(values_to_update):
            col_letter = column_to_letters(date_col_idx)
            cell_address = f"{col_letter}{schedule_row_idx + 12 + i}"
            schedule_wks.update_value(cell_address, value)
        
        cell_address = f"{col_letter}{schedule_row_idx + 7}"
        schedule_wks.update_value(cell_address, p_limit)
        cell_address = f"{col_letter}{schedule_row_idx + 8}"
        schedule_wks.update_value(cell_address, p_offset)